# System of non-linear equations - chemical equilibria
Author: Björn Dahlgren, Applied Physcial Chemistry, KTH Royal Insitiute of Technology

In this example we will study the equilibria between aqueous cupric ions and ammonia.
We will use a development branch of my experimental Python package [aqchem](github.com/bjodah/aqchem/pulls/4).

In [ ]:
from aqchem.chemistry import Solute, elements
from aqchem.equilibria import Equilibrium, EqSystemLog, EqSystemLin, composition_balance
from IPython.display import Latex, display
import periodictable
import matplotlib.pyplot as plt
%matplotlib inline
def show(s):  # convenience function
    display(Latex('$'+s+'$'))

Let's define our species with names and composition, we take help from the Python package [periodictable](https://pypi.python.org/pypi/periodictable):

In [ ]:
substances = Hp, OHm, NH4p, NH3, H2O, Cupp, CuNH31pp, CuNH32pp, CuNH33pp, CuNH34pp, CuNH35pp, Cu2OH2pp, CuOH3m, CuOH4mm, CuOH2 = [#, CuOHp, CuOH2, =
    Solute(n, latex_name=l, formula=periodictable.formula(n)) for n, l in [
        ('H{+}', 'H^+'), ('HO{-}', 'OH^-'), ('NH3 + H{+}', 'NH_4^+'),
        ('NH3', 'NH_3'), ('H2O', 'H_2O'), ('Cu{2+}', 'Cu^{2+}'), ('Cu{2+}NH3', 'Cu(NH_3)^{2+}'),
        ('Cu{2+}(NH3)2', 'Cu(NH_3)_2^{2+}'), ('Cu{2+}(NH3)3', 'Cu(NH_3)_3^{2+}'),
        ('Cu{2+}(NH3)4', 'Cu(NH_3)_4^{2+}'), ('Cu{2+}(NH3)5', 'Cu(NH_3)_5^{2+}'), 
        ('2Cu{2+} + 2HO{-}', 'Cu_2(OH)_2^{2+}'),
        ('Cu{2+} + 3HO{-}', 'Cu(OH)_3^-'), ('Cu{2+} + 4HO{-}', 'Cu(OH)_4^{2-}'),
        ('Cu{2+} + 2HO{-}', 'Cu(OH_2)(s)'),
    ]]
CuOH2.solid = True

Let's see how the Solutes are pretty-printed:

In [ ]:
show(', '.join([s.latex_name for s in substances]))

Let's define some initial concentrations. We will consider different amount of added ammonia in 10 mM solutions of $Cu^{2+}$:

In [ ]:
init_conc = {Hp: 1e-7, OHm: 1e-7, NH4p: 0, NH3: 1.0, Cupp: 1e-2, 
            CuNH31pp: 0, CuNH32pp: 0, CuNH33pp: 0, CuNH34pp: 0, CuNH35pp: 0,
            H2O: 55.5, Cu2OH2pp: 0, CuOH2: 0, CuOH3m: 0, CuOH4mm: 0}

Now, let us define the equilibria, data are from course material at Applied Physcial Chemistry, KTH Royal Insitiute of Technology.

In [ ]:
H2O_c = init_conc[H2O]
w_autop = Equilibrium({H2O: 1}, {Hp: 1, OHm: 1}, 10**-14/H2O_c)
NH4p_pr = Equilibrium({NH4p: 1}, {Hp: 1, NH3: 1}, 10**-9.26)
CuOH2_s = Equilibrium({CuOH2: 1}, {Cupp: 1, OHm: 2}, 10**-18.8)
CuOH_B3 = Equilibrium({CuOH2: 1, OHm: 1}, {CuOH3m: 1}, 10**-3.6)
CuOH_B4 = Equilibrium({CuOH2: 1, OHm: 2}, {CuOH4mm: 1}, 10**-2.7)
Cu2OH2 = Equilibrium({Cupp: 2, H2O: 2}, {Cu2OH2pp: 1, Hp: 2}, 10**-10.6 / H2O_c**2)
CuNH3_B1 = Equilibrium({CuNH31pp: 1}, {Cupp: 1, NH3: 1}, 10**-4.3)
CuNH3_B2 = Equilibrium({CuNH32pp: 1}, {Cupp: 1, NH3: 2}, 10**-7.9)
CuNH3_B3 = Equilibrium({CuNH33pp: 1}, {Cupp: 1, NH3: 3}, 10**-10.8)
CuNH3_B4 = Equilibrium({CuNH34pp: 1}, {Cupp: 1, NH3: 4}, 10**-13.0)
CuNH3_B5 = Equilibrium({CuNH35pp: 1}, {Cupp: 1, NH3: 5}, 10**-12.4)
equilibria = w_autop, NH4p_pr, CuNH3_B1, CuNH3_B2, CuNH3_B3, CuNH3_B4, CuNH3_B5, Cu2OH2, CuOH_B3, CuOH_B4, CuOH2_s

Let's see if we can print ``equilibria`` in a human-readable form:

In [ ]:
show(', '.join([s.latex_name for s in substances]))
show('~')
from math import log10
for eq in equilibria:
    ltx = eq.latex()
    show(ltx + '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~lgK = {0:12.5g}'.format(log10(eq.params)))

To keep our numerical treatment as simple as possible we will try to avoid representing
$Cu(OH)_2(s)$ explicitly (including the last solubility equilibrium). This is becuase the
system of equations change when precipitation sets in.
However, we do want to keep the second and third from last equilibria, therefore we rewrite
those using the last equiblibrium reaction to represent them only using dissolved species:

In [ ]:
new_eqs = CuOH2_s - CuOH_B3, CuOH2_s - CuOH_B4
new_eqs

Now it's time to exclude the solid species and replace the last three equilibria with our two new ones:

In [ ]:
#skip_subs, skip_eq = (4, 4) # (0, 0), (1, 1), (3, 3), (4, 4), (11, 9)
skip_subs, skip_eq = (1, 3)
simpl_subs = substances[:-skip_subs]
simpl_eq = equilibria[:-skip_eq] + new_eqs
simpl_c0 = {k: init_conc[k] for k in simpl_subs}

From the law of mass action we can from the equilbria and from the preservation of mass and charge formulate a non-linear system of equations:

In [ ]:
import sympy as sp
sp.init_printing()
lin_eqsys = EqSystemLin(simpl_eq, simpl_subs)
x, i = sp.symarray('x', lin_eqsys.ns), sp.symarray('i', lin_eqsys.ns)
lin_eqsys.f(x, i, ln=sp.log, exp=sp.exp)

It turns out that the success of the numerical root finding process for above system of equations is terrible sensitive on the choice of the initial guess. We therefore reformulate the equations in terms of the logarithm of the concentrations:

In [ ]:
eqsys = EqSystemLog(simpl_eq, simpl_subs)
f = eqsys.f(x, i, ln=sp.log, exp=sp.exp) #, rref_equil=True, rref_preserv=True)
f

We can take a peek on the jacobian of this vector:

In [ ]:
sp.Matrix(1, len(f), lambda _, q: f[q]).jacobian(x)

The preservation equations of mass and charge actually contain a redundant equation, so currently our system is over-determined:

In [ ]:
len(f), eqsys.ns

We could cast the preservation equations into reduced row echelon form (which would remove one equation), but for now we'll leave this be and rely on the Levenberg-Marquardt algorithm to solve our problem in a least-squares sense. (Levenberg-Marquardt uses QR-factorization internally for which it is acceptable to have overdetermined systems).

Let's solve the equations for our inital concentrations:

In [ ]:
C, sol = eqsys.root(simpl_c0, rref_preserv=True)
assert sol.success
C

Great, let's now vary the initial concentration of $NH_3$ and plot the equilibrium concentrations of our species:

In [ ]:
import numpy as np
plt.figure(figsize=(12,8))
Cout_logC, inits_logC, success = eqsys.solve_and_plot(
    simpl_c0, NH3, np.logspace(-3, 0, 100)
)
all(success)

But the above diagram is only true if we are below the solubility limit of our neglected $Cu(OH)_2(s)$. Let's plot the solubility product in the same diagram:

In [ ]:
sol_prod = Cout_logC[:, eqsys.as_substance_index(Cupp)]*Cout_logC[:, eqsys.as_substance_index(OHm)]**2
varied = inits_logC[:, eqsys.as_substance_index(NH3)]
plt.loglog(varied, sol_prod, label='[$%s$][$%s$]$^2$' % (Cupp.latex_name, OHm.latex_name))
plt.loglog(varied, Cout_logC[:, eqsys.as_substance_index(Hp)], ls=':', label='[$%s$]' % Hp.latex_name)
plt.loglog([varied[0], varied[-1]], [10**-18.8, 10**-18.8], 'k--', label='$K_{sp}(Cu(OH)_2(s))$')
plt.xlabel('[$NH_3$]')
_ = plt.legend()

We see that for a ammonia concentraion exceeding ~500-600 mM we would not precipitate $Cu(OH)_2(s)$ even though our pH is quite high (almost 12).

We have solved the above system of equations for the *logarithm* of the concentrations. How big are our absolute and relative errors compared to the linear system? Let's plot them:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20,8))
eqsys.plot_errors(Cout_logC, inits_logC, NH3, axes=axes)

Not bad. So the problem is essentially solved. For fun, let's see what the equation system looks like if we canonicalize it by transforming the equations for equibliria and the equations for the preservation relations to their respective reduced row echelon form:

In [ ]:
rf = eqsys.f(x, i, ln=sp.log, exp=sp.exp, rref_equil=True, rref_preserv=True)
rf

So the Jacobian should be considerably more diagonally dominant now:

In [ ]:
sp.Matrix(1, len(rf), lambda _, q: rf[q]).jacobian(x)

And let's see if this system converges as well (actually Powell's modefied version fails here so we are left with Levenberg-Marquardt)

In [ ]:
out = eqsys.solve_and_plot(simpl_c0, NH3, np.logspace(-3, 0, 100), roots_kwargs=dict(
        rref_equil=True, rref_preserv=True, method='lm'))